# 3-2 NeRF神经辐射场
  * [0 章节目标](#0-----)
  * [1 外观和形状表征](#1--------)
    + [1.1 形状表征](#11-----)
    + [1.2 外观表征](#12-----)
  * [2 神经辐射场NeRF](#2------nerf)
  * [3 代码演示](#3-----)
    + [3.1 依赖库安装](#31------)
    + [3.2 下载并准备数据集](#32---------)
    + [3.3 建立并启动viewer](#33------viewer)
    + [3.4 开始训练](#34-----)

## 0 章节目标
- 了解外观和形状表征，了解新视角合成任务
- 理解神经渲染原理（NeRF）

## 1 外观和形状表征
世界如此纷繁复杂？该以何种形式表示它们呢？我们可以将我们看到的世界解耦为两种形式，采用的是外观（appearance）和形状（shape/geometry）来表示一个物体或者场景。其中外观又细分为光照、材质等等。通过将组成世界的光照、几何、材质等元素进行解耦从而可以对场景进行参数化表示。计算机视觉中则是以简单粗暴的rgb色彩值来理解世界。

他们之间通过渲染和反渲染的方式进行转换。如下图所示，[来源](https://www.bilibili.com/video/BV1d34y1n7fn/?spm_id_from=333.788&vd_source=f1a43cfabe61c271b4df05136a41d18c)

<img alt="介绍" height="360" src="images/3-2_1-rendering-and-inverse-rendering.png" width="800"/>


其中渲染依赖于形状和外观的表征，并且这个过程需要可微。而反渲染可以粗浅地理解为通过同一个物体的多张不同角度的rgb图片反推出物体的几何或者外观参数。渲染是一个确定的过程，而反渲染目前的很多方法都是通过神经网络来猜。

<img alt="渲染过程" height="360" src="images/3-2_2-rendering.png" width="500"/>

### 1.1 形状表征
目前主流的如下图所示。其中，前三种方式称为显式的表征方式，它们一般便于编辑但消耗资源或者重建困难；后两种方式称为隐式的表征方式，它们一般重建方便但不易编辑，后两种方式都是通过运用神经网络来预测的，一个预测[符号距离函数](https://en.wikipedia.org/wiki/Signed_distance_function)的值，一个预测密度可见性的值。

<img alt="渲染过程" height="650" src="images/3-1_1-shape-representation.png" width="1200"/>

### 1.2 外观表征
外观是个很大的概念，一般又分为外观又细分为光照、材质等等。在我们得到物体和场景的几何和形状信息之后，简单来说，之后的流程可以简化成我们是把物体材质的信息对应贴在几何体上面，如下图所示。而nerf的方式则是将这个过程统一起来得到RGB的颜色，在接下来我们将详细介绍。



## 2 神经辐射场NeRF
NeRF (Neural Radiance Fields) [wiki](https://en.wikipedia.org/wiki/Neural_radiance_field)做的事情实质就是把形状和外观直接通过rgb值表示出来，不需要走传统图形学的那一套pipeline，这既是它的优点（可以很方便的表征场景）也是他的缺点（无法融入管线，使用不方便，编辑困难）。
首先，我们先直观的感受下nerf做的事情。

<img alt="介绍" height="800" src="./images/3-2_5-nerf-demo.gif" width="1400"/>

然后来看nerf的framework。

![nerf network](./images/3-2_3-nerf1.png)
其实它做的就是输入一些视角的图片，然后丢到神经网络中去预测新视角的图片。

具体而言就是输入一些空间中的坐标点的位置和方向角信息，之后通过神经网络来预测这些点的不透明度的值以及该点的颜色，最后通过rendering的方法来得到每个像素的颜色值，很多个像素最终组成整张图片的rgb值。
![nerf network](./images/3-2_4-nerf2.png)

知道这些之后基本就可以理解后面章节的内容了，如果想进一步了解原理的东西，可以参考以下资料：

> [NeRF: Representing Scenes as Neural Radiance Fields for View Synthesis](https://arxiv.org/pdf/2003.08934.pdf)
>
> [NeRF](https://www.matthewtancik.com/nerf)
>
> [都2022年了，我不允许你还不懂NeRF](https://zhuanlan.zhihu.com/p/569843149)
>
> [NeRF代码解读-相机参数与坐标系变换](https://zhuanlan.zhihu.com/p/593204605)


## 3 代码演示
> 代码参考：[nerfstudio-colab](https://colab.research.google.com/github/nerfstudio-project/nerfstudio/blob/main/colab/demo.ipynb)

### 3.1 依赖库安装

In [ ]:
%cd /content/
!pip install --upgrade pip
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

# Installing TinyCuda
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=1-7x7qQfB7bIw2zV4Lr6-yhvMpjXC84Q5&confirm=t"
!pip install tinycudann-1.7-cp310-cp310-linux_x86_64.whl

# Installing COLMAP
%cd /content/
!gdown "https://drive.google.com/u/0/uc?id=15WngFRNar_b8CaPR5R-hvQ3eAnlyk_SL&confirm=t"
!sudo apt-get install \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev
!unzip local.zip -d /usr/
!chmod +x /usr/local/bin/colmap

# Install nerfstudio
%cd /content/
!pip install git+https://github.com/nerfstudio-project/nerfstudio.git

### 3.2 下载并准备数据集

In [ ]:
#@markdown <h3>Pick the preset scene or upload your own images/video</h3>
import os
import glob
from google.colab import files
from IPython.core.display import display, HTML

scene = '\uD83D\uDE9C dozer' #@param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video', '🔺 upload Polycam data', '💽 upload your own Record3D data']
scene = ' '.join(scene.split(' ')[1:])

if scene == "upload Polycam data":
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    if len(uploaded.keys()) > 1:
        print("ERROR, upload a single .zip file when processing Polycam data")
    dataset_dir = [os.path.join(dir, f) for f in uploaded.keys()][0]
    !ns-process-data polycam --data $dataset_dir --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene == 'upload your own Record3D data':
    display(HTML('<h3>Zip your Record3D folder, and upload.</h3>'))
    display(HTML('<h3>More information on Record3D can be found <a href="https://docs.nerf.studio/en/latest/quickstart/custom_dataset.html#record3d-capture" target="_blank">here</a>.</h3>'))
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    record_3d_zipfile = preupload_datasets[0]
    !unzip $record_3d_zipfile -d /content/data/nerfstudio/custom_data
    custom_data_directory = glob.glob('/content/data/nerfstudio/custom_data/*')[0]
    !ns-process-data record3d --data $custom_data_directory --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene in ['upload your images', 'upload your own video']:
    display(HTML('<h3>Select your custom data</h3>'))
    display(HTML('<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>'))
    display(HTML('<p/>Note: This may take time, especially on higher resolution inputs, so we recommend to download dataset after creation.<p>'))
    !mkdir -p /content/data/nerfstudio/custom_data
    if scene == 'upload your images':
        !mkdir -p /content/data/nerfstudio/custom_data/raw_images
        %cd /content/data/nerfstudio/custom_data/raw_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/custom_data/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    if scene == 'upload your images':
        !ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/
    else:
        video_path = preupload_datasets[0]
        !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/

    scene = "custom_data"
else:
    %cd /content/
    !ns-download-data nerfstudio --capture-name=$scene

print("Data Processing Succeeded!")

### 3.3 建立并启动viewer

In [ ]:
%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt


with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")
from IPython import display
display.IFrame(src=url, height=800, width="100%")

### 3.4 开始训练

In [ ]:
%cd /content
if os.path.exists(f"data/nerfstudio/{scene}/transforms.json"):
    !ns-train nerfacto --viewer.websocket-port 7007 nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4
else:
    from IPython.core.display import display, HTML
    display(HTML('<h3 style="color:red">Error: Data processing did not complete</h3>'))
    display(HTML('<h3>Please re-run `Downloading and Processing Data`, or view the FAQ for more info.</h3>'))